In [3]:
import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.ml.linalg.Vectors


In [4]:
val denseVec = Vectors.dense(1.0,2.0,3.0)

denseVec: org.apache.spark.ml.linalg.Vector = [1.0,2.0,3.0]


In [8]:
val SparseVec= Vectors.sparse(10,Array(1,2,3,4,5),Array(11,12,13,14,15))

SparseVec: org.apache.spark.ml.linalg.Vector = (10,[1,2,3,4,5],[11.0,12.0,13.0,14.0,15.0])


# Spark ML Transformers

In [75]:
import org.apache.spark.ml.feature
import org.apache.spark.sql.functions._

import org.apache.spark.ml.feature
import org.apache.spark.sql.functions._


In [22]:
val df = spark.read.option("header","true").option("delimiter",",").option("inferSchema","true")
    .csv("C:\\Users\\vitormendes\\Documents\\Coursera\\Untitled Folder\\Week 2\\home_data.csv")

df: org.apache.spark.sql.DataFrame = [id: bigint, date: string ... 19 more fields]


In [26]:
df.select($"price").show(2)

+------+
| price|
+------+
|221900|
|538000|
+------+
only showing top 2 rows



In [31]:
val df2 = df.select($"price".cast("double"))

df2: org.apache.spark.sql.DataFrame = [price: double]


In [48]:
val priceAssembler = new feature.VectorAssembler()
priceAssembler.setInputCols(Array("price")).setOutputCol("features")

priceAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_a509675bc62e
res31: priceAssembler.type = vecAssembler_a509675bc62e


In [50]:
val priceAssemblerDF = priceAssembler.transform(df.select($"price"))

priceAssemblerDF: org.apache.spark.sql.DataFrame = [price: int, features: vector]


In [57]:
val scaler_minmax = new feature.MinMaxScaler().setMin(1).setMax(50)
scaler_minmax.setInputCol("features").setOutputCol("price_feature")

scaler_minmax: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_c3d6e87337b7
res38: scaler_minmax.type = minMaxScal_c3d6e87337b7


In [58]:
val scaled = scaler_minmax.fit(priceAssemblerDF).transform(priceAssemblerDF)

scaled: org.apache.spark.sql.DataFrame = [price: int, features: vector ... 1 more field]


In [103]:
val scaled2 = scaled.select(
    expr("*"),
    when($"price" > 200000 , $"price").otherwise(null).cast("double").alias("a")
)

scaled2: org.apache.spark.sql.DataFrame = [price: int, features: vector ... 2 more fields]


In [104]:
scaled2.agg(mean($"price")).show()

+-----------------+
|       avg(price)|
+-----------------+
|540088.1419053348|
+-----------------+



In [110]:
val imputer = new feature.Imputer()
imputer
    .setInputCols(Array("a"))
    .setOutputCols(Array("b"))
    .setStrategy("mean")

imputer: org.apache.spark.ml.feature.Imputer = imputer_a2024ae49650
res74: imputer.type = imputer_a2024ae49650


In [112]:
val scaled3 = imputer.fit(scaled2).transform(scaled2)

scaled3: org.apache.spark.sql.DataFrame = [price: int, features: vector ... 3 more fields]


In [113]:
scaled3.show(5)

+------+----------+--------------------+--------+-----------------+
| price|  features|       price_feature|       a|                b|
+------+----------+--------------------+--------+-----------------+
|221900|[221900.0]|[1.9440131147540982]|221900.0|         221900.0|
|538000|[538000.0]| [3.975344262295082]|538000.0|         538000.0|
|180000|[180000.0]|[1.6747540983606557]|    null|555145.9780473715|
|604000|[604000.0]| [4.399475409836066]|604000.0|         604000.0|
|510000|[510000.0]| [3.795409836065574]|510000.0|         510000.0|
+------+----------+--------------------+--------+-----------------+
only showing top 5 rows



# R Formula

In [133]:
val students = Seq(
    (1,"Vitor","tall",9.0),
    (2,"Henrique","tall",8.7),
    (3,"Akira","short",5.5),
    (1,"Vitor","short",6.2)
).toDF("id","name","size","grade")

students: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


In [134]:
students.show()

+---+--------+-----+-----+
| id|    name| size|grade|
+---+--------+-----+-----+
|  1|   Vitor| tall|  9.0|
|  2|Henrique| tall|  8.7|
|  3|   Akira|short|  5.5|
|  1|   Vitor|short|  6.2|
+---+--------+-----+-----+



In [124]:
val formula = new feature.RFormula()
formula.setFormula("grade ~ size")

formula: org.apache.spark.ml.feature.RFormula = RFormula(grade ~ size) (uid=rFormula_11ebaf888f83)
res81: formula.type = RFormula(grade ~ size) (uid=rFormula_11ebaf888f83)


In [125]:
formula.fit(students).transform(students).show()

+---+--------+-----+-----+--------+-----+
| id|    name| size|grade|features|label|
+---+--------+-----+-----+--------+-----+
|  1|   Vitor| tall|  9.0|   [1.0]|  9.0|
|  2|Henrique| tall|  8.7|   [1.0]|  8.7|
|  3|   Akira|short|  5.5|   [0.0]|  5.5|
+---+--------+-----+-----+--------+-----+



In [126]:
import feature.{StringIndexer,OneHotEncoder}

import feature.{StringIndexer, OneHotEncoder}


In [152]:
val string_indexer = new StringIndexer().setInputCol("size").setOutputCol("size_indexer").fit(students).transform(students)
val hot_encoder = new OneHotEncoder().setInputCol("size_indexer").setOutputCol("size_binarization").setDropLast(false)

string_indexer: org.apache.spark.sql.DataFrame = [id: int, name: string ... 3 more fields]
hot_encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_519d0993cb71


In [153]:
val students_ml = hot_encoder.transform(string_indexer)

students_ml: org.apache.spark.sql.DataFrame = [id: int, name: string ... 4 more fields]


In [154]:
students_ml.show()

+---+--------+-----+-----+------------+-----------------+
| id|    name| size|grade|size_indexer|size_binarization|
+---+--------+-----+-----+------------+-----------------+
|  1|   Vitor| tall|  9.0|         1.0|    (2,[1],[1.0])|
|  2|Henrique| tall|  8.7|         1.0|    (2,[1],[1.0])|
|  3|   Akira|short|  5.5|         0.0|    (2,[0],[1.0])|
|  1|   Vitor|short|  6.2|         0.0|    (2,[0],[1.0])|
+---+--------+-----+-----+------------+-----------------+



In [150]:
students.groupBy("name").agg(
    max($"id").alias("id"),collect_set($"size").alias("size"),collect_set($"grade").alias("grade")
).show()

+--------+---+-------------+----------+
|    name| id|         size|     grade|
+--------+---+-------------+----------+
|Henrique|  2|       [tall]|     [8.7]|
|   Akira|  3|      [short]|     [5.5]|
|   Vitor|  1|[short, tall]|[9.0, 6.2]|
+--------+---+-------------+----------+

